# Full-waveform inversion using automatic differentiation

Full-waveform inversion (FWI) is a high-resolution seismic technique used to estimate the physical parameters in a subsurface region. It is a wave-equation-based technique that searches for an optimal match between observed and computed data. The former is recorded by receivers in the field, whereas the latter consists of computed estimates of propagated waves emitted by a specified wave source. The observed data at the receivers are subject to the influences of the subsurface medium, while waves propagate from the source. Synthetic data can be generated by propagating the source waves in an estimated medium, and therefore, the minimisation of the differences between the observed and synthetic data at the receivers is a methodology for seeking the medium properties of a region. 

The data difference is traditionally measured by a least square misfit function (Tarantola, 1984): 
$$
    I (m)\equiv \frac{1}{2} \sum_{i=1}^ {N_r} \int_{\tau} \left(u(v_p,\check{\mathbf{x}},t)- u^{obs}(v_p,\check{\mathbf{x}},t)\right)^2 \text{d} t . 
$$

The data functions, $u = u(v_p, \check{\mathbf{x}},t)$ and $u_{obs} = u_{obs}(v_p,\check{\mathbf{x}},t)$, are respectively the predicted and observed data, both recorded at a finite number of receivers ($N_r$), located at the point positions $\check{\mathbf{x}} \in \Omega_{0}$, in a time interval $\tau\equiv[t_0, t_f]\subset \mathbb{R}$, where $t_0$ is the initial time and $t_f$ is the final time. The spatial domain of interest is set as $\Omega_{0}$.

## Wave equation
In computational procedures, FWI consist of solving a wave equation in a limited domain to obtain the predicted data $u$ at the receivers. In this example, we consider the acoustic wave equation
$$
    \frac{1}{v_p^2(\mathbf{x})} \frac{\partial^2 u}{\partial t^2}(\mathbf{x},t)-\frac{\partial^2 u}{\partial \mathbf{x}^2} = f(\mathbf{x},t),
$$
where $c(\mathbf{x}):\Omega_{0}\rightarrow \mathbb{R}$ is the pressure wave ($P$-wave) velocity, which is assumed to be piecewise-constant and positive. 

As we mentioned above, the wave propagate from a source. Here, the source of waves given by a Ricker Wavelet \cite{ricker1940form}.

In this example, we consider a two dimensional domain where we want to estimate the parameter $v_p$ in a physical domain with the lenght of $1$ km ($L_x = 1km$) and the depth of $1$ km ($L_z = 1km$). We need to extend this domain since on limited domain leads spurious wave reflections to appear, which means that nonphysical information. To tackle this problem, we apply the damping boundary conditions in the extended domain. 

The domain extension is of $0.2 km$ in each direction. Therefore, the total domain size is $L_x = 1.4$ and $L_z = 1.2$ km in each direction.

In [ ]:
import firedrake as fd
Lx = 1.4
Lz = 1.2
mesh = fd.SquareMesh(Lx, Lz, 50, 50)

Initially, we consider a source located at the center of the domain and to the free surface. Thus, the source position is defined as follows:

In [ ]:
src_pos = [0.6, 0.1]

We consider the number of receivers $N_r = 10$. The are uniformly distributed close to the free surface boundary. Hence, on considering this assumptions, we can define the receiver locations as follows:

In [ ]:
import numpy as np
z_pos = 0.1
num_receivers = 10
receivers = np.linspace((0.3, z_pos), (0.9, z_pos), num_receivers)

The code below sets Ricker wavelet peak of frequency, the time step and the total time of simulation.

In [ ]:
T = 1.0
dt = 0.001
freq = 7.0

The acoustic wave equation is a second-order hyperbolic partial differential equation that describes the propagation of acoustic waves through a medium. 

The function `wave_eq_solver` define the acoustic wave equation solver. For additional details regarding the acoustic wave equation, please refer to the following link: https://www.firedrakeproject.org/demos/higher_order_mass_lumping.py.html

In [ ]:
import finat
def wave_eq_solver(c, source_f):
    V = fd.FunctionSpace(mesh, "KMV", 2)
    u = fd.TrialFunction(V)
    v = fd.TestFunction(V)

    u_np1 = fd.Function(V)  # timestep n+1
    u_n = fd.Function(V)    # timestep n
    u_nm1 = fd.Function(V)  # timestep n-1

    # quadrature rule for lumped mass matrix
    quad_rule = finat.quadrature.make_quadrature(V.finat_element.cell, V.ufl_element().degree(), "KMV")
    dxlump=fd.dx(scheme=quad_rule)
    # time discretisation/mass matrix
    m = 1 / (c * c) * (u - 2.0 * u_n + u_nm1) / fd.Constant(dt * dt) * v * dxlump
    # stiffness matrix
    a = c * c * fd.dot(fd.grad(u_n), fd.grad(v)) * fd.dx
    # wave source
    f = source_f * v * dxlump
    F = m + a - f
    lhs_ = fd.lhs(F)
    rhs_ = fd.rhs(F)

    lin_var = fd.LinearVariationalProblem(lhs_, rhs_, u_np1)
    solver = fd.LinearVariationalSolver(lin_var, 
                                        solver_parameters=
                                        {"ksp_type": "preonly", "pc_type": "jacobi"})
    
    return solver, u_np1, u_n, u_nm1

## Receivers data
Firedrake is enable ...

The code below define the function space of zero order discontinuous Lagrange polynomials, and an Interpolator of the pressure wave solution at the receiver positions.

In [ ]:
def p0dg_interpolation(wave_solution):
    P = fd.VectorFunctionSpace(receivers, "DG", 0)
    rec_interpolator = fd.Interpolator(wave_solution, P)
    return P, rec_interpolator

## Wave Source data

The following two functions are used to inject the Ricker wavelet source into the domain. We create a time-varying function to model the time evolution of the Ricker wavelet:

In [ ]:
import numpy as np

def source(constant, t, amp=1.0):
    # Shift in time so the entire wavelet is injected
    t = t - (np.sqrt(6.0) / (np.pi * freq))
    wavelet = amp * (
        1.0 - (1.0 / 2.0) * (2.0 * np.pi * freq) * (2.0 * np.pi * freq) * t * t
    )
    x, y = fd.SpatialCoordinate(mesh)

    # Guassian kernel with a standard deviation of 2,000
    delta = fd.exp(-2000 * ((x - src_pos[0]) ** 2 + (y - src_pos[1]) ** 2))
    
    return delta*constant.assign(wavelet) 

In [ ]:
def make_vp_circle(vp_guess=False):
    """creating velocity models"""
    x, z = fd.SpatialCoordinate(mesh)
    if vp_guess:
        vp = fd.Function(V).interpolate(1.5 + 0.0 * x)
    else:
        vp = fd.Function(V).interpolate(
            2.5
            + 1 * fd.tanh(100 * (0.125 - fd.sqrt((x - 0.5) ** 2 + (z - 0.5) ** 2)))
        )
    # fire.File("vp.pvd").write(vp)
 
    return vp

In [ ]:
rec_data = []
c = fd.Constant(1.5)
solver, u_np1, u_n, u_nm1 = wave_eq_solver(c, source)
ricker = fd.Constant(0.0)
P, rec_interpolator = p0dg_interpolation(u_np1)
t = 0
while t < T:
    f = source(ricker, t)
    # Call the solver object.
    solver.solve()

    # Exchange the solution at the two time-stepping levels.
    u_nm1.assign(u_n)
    u_n.assign(u_np1)

    rec = fd.Function(P, name="rec")
    rec_interpolator.interpolate(output=rec)
    rec_data.append(rec)


In [ ]:
import firedrake as fd
def cost_function(u, u_obs):
    J = fd.assemble(0.5*fd.inner(u - u_obs, u - u_obs) * fd.dx)

In [ ]:
def true_data(c):
    solver, u_n, u_nm1, u_np1, ricker = wave_eq_solver(c)
    step = 0
    rec_data = []
    P = fd.VectorFunctionSpace(receivers, "DG", 0)
    rec_interpolator = fd.Interpolator(u_np1, P)
    while t < T:
        ricker.assign(RickerWavelet(t, freq))
        # Call the solver object.
        solver.solve()

        # Exchange the solution at the two time-stepping levels.
        u_nm1.assign(u_n)
        u_n.assign(u_np1)

        rec = fd.Function(P, name="rec")
        rec_interpolator.interpolate(output=rec)
        rec_data.append(rec)
        
        # Increment the time and print the elapsed time every 10 steps.
        t += dt
        step += 1
        if step % 10 == 0:
            print("Elapsed time is: "+str(t))
        
    return rec_data

true_rec_data = true_data(c)

In [ ]:
def guess_solver(c, true_data):
    solver, u_n, u_nm1, u_np1, ricker = wave_eq_solver(c)
    step = 0
    rec_data = []
    P = fd.VectorFunctionSpace(receivers, "DG", 0)
    rec_interpolator = fd.Interpolator(u_np1, P)
    J = 0.0 
    while t < T:
        ricker.assign(RickerWavelet(t, freq))
        # Call the solver object.
        solver.solve()

        # Exchange the solution at the two time-stepping levels.
        u_nm1.assign(u_n)
        u_n.assign(u_np1)
        
        rec = fd.Function(P, name="rec")
        rec_interpolator.interpolate(output=rec)
        rec_data.append(rec)
        J += cost_function(rec_data, true_data[step])
        
    return J

In [ ]:
import firedrake.adjoint as fd_adj


fd_adj.ReducedFunctional(J, c_guess)
fd_adj.minimize(guess_solver, method="L-BFGS-B", options={"disp": True})